In [ ]:
# !git clone https://github.com/ankitsunil530/blood-cell-classification.git
# %cd blood-cell-classification

fatal: destination path 'blood-cell-classification' already exists and is not an empty directory.
/content/blood-cell-classification/blood-cell-classification


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Blood Cell Classification**

## Data loading and preprocessing

### Subtask:
Load the blood cell dataset and preprocess the images for use with the vision transformer and Performer models. This will involve resizing, normalization, and splitting the data into training, validation, and test sets.


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


In [3]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.9/753.9 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.3 MB/s eta 0:00:00


In [3]:
import os
import time
import glob
import random
from pathlib import Path
from typing import List

import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.transforms.functional as F

import mlflow
import mlflow.pytorch

ModuleNotFoundError: No module named 'mlflow'

In [4]:
pip install mlflow

^C
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mysql-connector-python 8.2.0 requires protobuf<=4.21.12,>=4.21.1, but you have protobuf 6.33.1 which is incompatible.
pyopenssl 23.0.0 requires cryptography<40,>=38.0.0, but you have cryptography 46.0.3 which is incompatible.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.
tensorflow-intel 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.1 which is incompatible.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached mlflow-3.6.0-py3-none-any.whl.metadata (31 kB)
  Using cached mlflow_skinny-3.6.0-py3-none-any.whl.metadata (31 kB)
  Using cached mlflow_tracing-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached flask_cors-6.0.1-py3-none-any.whl.metadata (5.3 kB)
  Using cached cryptography-46.0.3-cp311-abi3-win_amd64.whl.metadata (5.7 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached huey-2.5.4-py3-none-any.whl.metadata (4.6 kB)
  Using cached waitress-3.0.2-py3-none-any.whl.metadata (5.8 kB)
  Using cached databricks_sdk-0.73.0-py3-none-any.whl.metadata (40 kB)
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached opentelemetry_api-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_proto-1.38.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_sdk-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached python_dotenv-1.2

# GAN Model

In [14]:
# train_gan_corrected.py
import os
import time
import glob
import random
from pathlib import Path
from typing import List

import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.transforms.functional as F

import mlflow
import mlflow.pytorch

# --------------------
# CONFIG / HYPERPARAMS (tweak as needed)
# --------------------
# Set this to the parent "images" folder that contains TRAIN, TEST, TEST_SIMPLE
DATASET_PATH = r"/content/drive/MyDrive/Deep Learning Lab/Blood_Cells_Dataset/Blood_Cells_Dataset/dataset2-master/dataset2-master/images"

OUT_DIR = "gan_outputs"
MODEL_DIR = "models"
EXPERIMENT = "Blood-Cell-GAN"
RUN_NAME = "DCGAN-corrected-run"

IMG_SIZE = 128          # 64 for faster experiments, 128 gives better visuals
BATCH_SIZE = 32         # reduce if OOM on GPU
Z_DIM = 100
NUM_EPOCHS = 20
LR = 2e-4
BETA1 = 0.5
NUM_WORKERS = 2         # use 0 on Windows if you see DataLoader worker issues
SAMPLE_EVERY = 5        # save sample & checkpoint every N epochs
LOG_BATCH_EVERY = 200   # log batch metrics to MLflow every N batches

# Label smoothing
REAL_LABEL = 0.9
FAKE_LABEL = 0.0

# Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True  # may speed up for fixed-size inputs

os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

# --------------------
# Custom Dataset: collects images from TRAIN/TEST/TEST_SIMPLE -> class folders
# --------------------
class CombinedImageDataset(Dataset):
    def __init__(self, images_root: str, transform=None):
        """
        images_root should contain folders like TRAIN, TEST, TEST_SIMPLE.
        Each of those contains class folders (e.g., NEUTROPHIL, LYMPHOCYTE, ...).
        """
        self.images_root = Path(images_root)
        self.transform = transform

        # folders to scan
        candidates = ["TRAIN", "TEST", "TEST_SIMPLE"]
        self.samples: List[Path] = []
        self.classes = set()

        for c in candidates:
            folder = self.images_root / c
            if not folder.exists():
                continue
            for class_dir in folder.iterdir():
                if class_dir.is_dir():
                    self.classes.add(class_dir.name)
                    # collect image paths
                    exts = ("*.jpg", "*.jpeg", "*.png", "*.bmp")
                    for ext in exts:
                        self.samples += list(class_dir.glob(ext))

        self.samples = [p for p in self.samples if p.is_file()]
        self.samples.sort()  # reproducible order
        self.classes = sorted(list(self.classes))
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}

        # build label mapping by path -> class index
        self.labels = []
        for p in self.samples:
            # find the class folder name by walking up until we hit the class directory
            # sample path: .../images/TRAIN/NEUTROPHIL/img.jpg  -> we want 'NEUTROPHIL'
            # find parent in path components that matches a class name
            cls_name = None
            for parent in p.parents:
                if parent.name in self.class_to_idx:
                    cls_name = parent.name
                    break
            if cls_name is None:
                # fallback: use folder immediate parent
                cls_name = p.parent.name
            self.labels.append(self.class_to_idx.get(cls_name, 0))

        if len(self.samples) == 0:
            raise RuntimeError(f"No images found under {images_root}. Check DATASET_PATH and structure.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        p = self.samples[idx]
        img = Image.open(p).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

# --------------------
# Transforms & DataLoader
# --------------------
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),  # scale to [-1, 1]
])

dataset = CombinedImageDataset(DATASET_PATH, transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True,
                        num_workers=NUM_WORKERS, pin_memory=True, drop_last=True)

print("Dataset classes:", dataset.classes)
print("Dataset size:", len(dataset))

# --------------------
# DCGAN Model definitions (Generator & Discriminator)
# --------------------
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

class Generator(nn.Module):
    def __init__(self, z_dim=100, ngf=64, nc=3):
        super().__init__()
        self.net = nn.Sequential(
            # z -> ngf*16 x 4 x 4
            nn.ConvTranspose2d(z_dim, ngf*16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*16), nn.ReLU(True),

            # -> ngf*8 x 8 x 8
            nn.ConvTranspose2d(ngf*16, ngf*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*8), nn.ReLU(True),

            # -> ngf*4 x 16 x 16
            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4), nn.ReLU(True),

            # -> ngf*2 x 32 x 32
            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2), nn.ReLU(True),

            # -> ngf x 64 x 64
            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf), nn.ReLU(True),

            # -> nc x 128 x 128
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        return self.net(z)

class Discriminator(nn.Module):
    def __init__(self, nc=3, ndf=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False), nn.BatchNorm2d(ndf*2), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False), nn.BatchNorm2d(ndf*4), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False), nn.BatchNorm2d(ndf*8), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*8, ndf*16, 4, 2, 1, bias=False), nn.BatchNorm2d(ndf*16), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*16, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x).view(-1, 1).squeeze(1)

# instantiate models
netG = Generator(z_dim=Z_DIM).to(DEVICE)
netD = Discriminator().to(DEVICE)
netG.apply(weights_init)
netD.apply(weights_init)

# --------------------
# Loss and optimizers
# --------------------
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=LR, betas=(BETA1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=LR, betas=(BETA1, 0.999))

fixed_noise = torch.randn(64, Z_DIM, 1, 1, device=DEVICE)

# --------------------
# MLflow experiment
# --------------------
mlflow.set_experiment(EXPERIMENT)
run = mlflow.start_run(run_name=RUN_NAME)
mlflow.log_params({
    "img_size": IMG_SIZE, "batch_size": BATCH_SIZE, "z_dim": Z_DIM,
    "lr": LR, "beta1": BETA1, "epochs": NUM_EPOCHS, "device": str(DEVICE)
})

print("Starting training on device:", DEVICE)
print("Total steps per epoch:", len(dataloader))

# --------------------
# Training loop (improved)
# --------------------
iters = 0
try:
    for epoch in range(1, NUM_EPOCHS + 1):
        epoch_start = time.time()
        running_D_loss = 0.0
        running_G_loss = 0.0

        for i, (data, _) in enumerate(dataloader):
            netD.zero_grad()

            # ------------------
            # Train D on real
            # ------------------
            real_images = data.to(DEVICE)
            b_size = real_images.size(0)
            real_labels = torch.full((b_size,), REAL_LABEL, dtype=torch.float, device=DEVICE)
            output_real = netD(real_images)
            errD_real = criterion(output_real, real_labels)
            errD_real.backward()
            D_x = output_real.mean().item()

            # ------------------
            # Train D on fake
            # ------------------
            noise = torch.randn(b_size, Z_DIM, 1, 1, device=DEVICE)
            fake_images = netG(noise)
            fake_labels = torch.full((b_size,), FAKE_LABEL, dtype=torch.float, device=DEVICE)
            output_fake = netD(fake_images.detach())
            errD_fake = criterion(output_fake, fake_labels)
            errD_fake.backward()
            D_G_z1 = output_fake.mean().item()

            # Update D
            optimizerD.step()

            # ------------------
            # Train G
            # ------------------
            netG.zero_grad()
            # want generator to produce images labeled as real (smoothed)
            gen_labels = torch.full((b_size,), REAL_LABEL, dtype=torch.float, device=DEVICE)
            output_for_G = netD(fake_images)
            errG = criterion(output_for_G, gen_labels)
            errG.backward()
            D_G_z2 = output_for_G.mean().item()
            optimizerG.step()

            # accumulate
            running_D_loss += (errD_real.item() + errD_fake.item())
            running_G_loss += errG.item()

            # occasional logging to MLflow to avoid I/O overhead
            if iters % LOG_BATCH_EVERY == 0:
                mlflow.log_metric("D_loss_batch", (errD_real.item() + errD_fake.item()), step=iters)
                mlflow.log_metric("G_loss_batch", errG.item(), step=iters)

            iters += 1

        avg_D = running_D_loss / len(dataloader)
        avg_G = running_G_loss / len(dataloader)
        epoch_time = time.time() - epoch_start
        print(f"Epoch [{epoch}/{NUM_EPOCHS}]  D_loss: {avg_D:.4f}  G_loss: {avg_G:.4f}  time: {epoch_time:.1f}s")

        mlflow.log_metric("D_loss_epoch", avg_D, step=epoch)
        mlflow.log_metric("G_loss_epoch", avg_G, step=epoch)

        # save checkpoints & samples periodically
        if epoch % SAMPLE_EVERY == 0 or epoch == 1 or epoch == NUM_EPOCHS:
            g_path = os.path.join(MODEL_DIR, f"netG_epoch{epoch}.pth")
            d_path = os.path.join(MODEL_DIR, f"netD_epoch{epoch}.pth")
            torch.save(netG.state_dict(), g_path)
            torch.save(netD.state_dict(), d_path)
            # save as mlflow artifacts (model weights + sample grid)
            mlflow.log_artifact(g_path)
            mlflow.log_artifact(d_path)

            with torch.no_grad():
                fake_samples = netG(fixed_noise).detach().cpu()
            grid = utils.make_grid(fake_samples, padding=2, normalize=True)
            sample_path = os.path.join(OUT_DIR, f"sample_epoch{epoch}.png")
            utils.save_image(grid, sample_path)
            mlflow.log_artifact(sample_path)
            print(f"Saved sample and checkpoints for epoch {epoch}")

    print("Training finished!")

except Exception as e:
    print("Training interrupted with error:", e)
    raise

finally:
    # make sure run is ended
    mlflow.end_run()


Dataset classes: ['EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL']
Dataset size: 12515
Starting training on device: cuda
Total steps per epoch: 391
Epoch [1/20]  D_loss: 0.5808  G_loss: 49.1429  time: 83.5s
Saved sample and checkpoints for epoch 1
Epoch [2/20]  D_loss: 0.3438  G_loss: 46.3876  time: 75.5s
Epoch [3/20]  D_loss: 0.3303  G_loss: 43.5043  time: 75.7s
Epoch [4/20]  D_loss: 1.2128  G_loss: 4.9556  time: 75.3s
Epoch [5/20]  D_loss: 1.0734  G_loss: 2.5391  time: 74.6s
Saved sample and checkpoints for epoch 5
Epoch [6/20]  D_loss: 1.0945  G_loss: 2.2189  time: 76.5s
Epoch [7/20]  D_loss: 1.1901  G_loss: 1.9315  time: 74.8s
Epoch [8/20]  D_loss: 1.2475  G_loss: 1.6270  time: 75.3s
Epoch [9/20]  D_loss: 1.2469  G_loss: 1.6032  time: 75.4s
Epoch [10/20]  D_loss: 1.2174  G_loss: 1.6800  time: 74.6s
Saved sample and checkpoints for epoch 10
Epoch [11/20]  D_loss: 1.2133  G_loss: 1.6880  time: 76.1s
Epoch [12/20]  D_loss: 1.2095  G_loss: 1.7421  time: 74.5s
Epoch [13/20]  D_loss

In [2]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

class Generator(nn.Module):
    def __init__(self, z_dim=100, ngf=64, nc=3):
        super().__init__()
        self.net = nn.Sequential(
            # z -> ngf*16 x 4 x 4
            nn.ConvTranspose2d(z_dim, ngf*16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*16), nn.ReLU(True),

            # -> ngf*8 x 8 x 8
            nn.ConvTranspose2d(ngf*16, ngf*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*8), nn.ReLU(True),

            # -> ngf*4 x 16 x 16
            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4), nn.ReLU(True),

            # -> ngf*2 x 32 x 32
            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2), nn.ReLU(True),

            # -> ngf x 64 x 64
            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf), nn.ReLU(True),

            # -> nc x 128 x 128
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        return self.net(z)

class Discriminator(nn.Module):
    def __init__(self, nc=3, ndf=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False), nn.BatchNorm2d(ndf*2), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False), nn.BatchNorm2d(ndf*4), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False), nn.BatchNorm2d(ndf*8), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*8, ndf*16, 4, 2, 1, bias=False), nn.BatchNorm2d(ndf*16), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*16, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x).view(-1, 1).squeeze(1)

# instantiate models
netG = Generator(z_dim=Z_DIM).to(DEVICE)
netD = Discriminator().to(DEVICE)
netG.apply(weights_init)
netD.apply(weights_init)

NameError: name 'nn' is not defined

In [ ]:
import torch
import os

# Create folder to save models
os.makedirs("models", exist_ok=True)

# Save Generator and Discriminator weights
torch.save(generator.state_dict(), "models/gan_generator.pth")
torch.save(discriminator.state_dict(), "models/gan_discriminator.pth")

print("✅ GAN models saved successfully!")
